In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
plt.style.use('fivethirtyeight')
import time
import progressbar

In [2]:
csv = 'lib/clean_text.csv'
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

/root/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,text,target
0,awww bummer shoulda got david carr third day,0
1,upset updat facebook text might cri result sch...,0
2,dive mani time ball manag save rest go bound,0
3,whole bodi feel itchi like fire,0
4,behav mad see,0


In [3]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620947 entries, 0 to 1620946
Data columns (total 2 columns):
text      1620947 non-null object
target    1620947 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.7+ MB


In [4]:
x = my_df.text
y = my_df.target

In [5]:
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [6]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [7]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [8]:
len(all_x_w2v)

1620947

In [9]:
cores=multiprocessing.cpu_count()

In [10]:
widgets = [progressbar.Percentage(),progressbar.Bar()," Translated : ",progressbar.Counter(),"  ",progressbar.ETA()]
bar = progressbar.ProgressBar(widgets=widgets, max_value=len(my_df.index))

In [11]:
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs

# DBOW

In [12]:
model_ug_dbow = Doc2Vec(dm=0, vector_size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

100%|██████████| 1620947/1620947 [00:00<00:00, 2834958.87it/s]


In [13]:
%%time
for epoch in range(30):
    model_ug_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dbow.alpha -= 0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha

100%|██████████| 1620947/1620947 [00:00<00:00, 2672758.73it/s]


CPU times: user 45min 18s, sys: 7min 28s, total: 52min 47s
Wall time: 42min 40s


In [14]:
model_ug_dbow.most_similar('good')

/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('cobb', 0.4523930549621582),
 ('nycc', 0.3903326690196991),
 ('webcast', 0.3788723349571228),
 ('faaaaast', 0.3770343065261841),
 ('ssiiccaa', 0.375257670879364),
 ('extinguish', 0.37267398834228516),
 ('federico', 0.3608899712562561),
 ('mainit', 0.3596012592315674),
 ('vil', 0.35707786679267883),
 ('gourock', 0.3530207872390747)]

In [15]:
train_vecs_dbow = get_vectors(model_ug_dbow, x_train, 100)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_validation, 100)
clf = LogisticRegression()
clf.fit(train_vecs_dbow, y_train)
clf.score(validation_vecs_dbow, y_validation)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7128138688382997

In [ ]:
model_ug_dbow.save('lib/d2v_dbow_eng')
model_ug_dbow = Doc2Vec.load('lib/d2v_dbow_eng')
model_ug_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

# DMC

In [ ]:
model_ug_dmc = Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])

100%|██████████| 1620947/1620947 [00:00<00:00, 2969791.85it/s]


In [ ]:
%%time
for epoch in range(30):
    model_ug_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmc.alpha -= 0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha

100%|██████████| 1620947/1620947 [00:00<00:00, 2719732.78it/s]


CPU times: user 1h 4min 25s, sys: 8min 21s, total: 1h 12min 47s
Wall time: 48min 5s


In [ ]:
model_ug_dmc.most_similar('good')

/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('gooooood', 0.7312248349189758),
 ('gooooooood', 0.6687590479850769),
 ('gooood', 0.66729336977005),
 ('goooooood', 0.6630001068115234),
 ('gud', 0.6496378183364868),
 ('goood', 0.6460071206092834),
 ('great', 0.6389926671981812),
 ('rtwt', 0.6371935606002808),
 ('chuckyoutuesday', 0.6360832452774048),
 ('gd', 0.6342563033103943)]

In [ ]:
train_vecs_dmc = get_vectors(model_ug_dmc, x_train, 100)
validation_vecs_dmc = get_vectors(model_ug_dmc, x_validation, 100)
clf = LogisticRegression()
clf.fit(train_vecs_dmc, y_train)
clf.score(validation_vecs_dmc, y_validation)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6342155592572027

In [ ]:
model_ug_dmc.save('lib/d2v_dmc_eng')
model_ug_dmc = Doc2Vec.load('lib/d2v_dmc_eng')

# DMM

In [ ]:
model_ug_dmm = Doc2Vec(dm=1, dm_mean=1, size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmm.build_vocab([x for x in tqdm(all_x_w2v)])

/root/anaconda3/envs/py36/lib/python3.6/site-packages/gensim/models/doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 1620947/1620947 [00:01<00:00, 931153.09it/s] 


In [ ]:
%%time
for epoch in range(30):
    model_ug_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmm.alpha -= 0.002
    model_ug_dmm.min_alpha = model_ug_dmm.alpha

100%|██████████| 1620947/1620947 [00:00<00:00, 2825223.30it/s]


CPU times: user 1h 33min 15s, sys: 35min 27s, total: 2h 8min 43s
Wall time: 1h 41min 46s


In [ ]:
model_ug_dmm.most_similar('good')

/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('go', 0.9642423987388611),
 ('like', 0.9642223119735718),
 ('lol', 0.9617702960968018),
 ('get', 0.9615004062652588),
 ('work', 0.9601917266845703),
 ('think', 0.9581536054611206),
 ('today', 0.9544107913970947),
 ('love', 0.9539278745651245),
 ('know', 0.9534299969673157),
 ('great', 0.9534275531768799)]

In [ ]:
train_vecs_dmm = get_vectors(model_ug_dmm, x_train, 100)
validation_vecs_dmm = get_vectors(model_ug_dmm, x_validation, 100)
clf = LogisticRegression()
clf.fit(train_vecs_dmm, y_train)
clf.score(validation_vecs_dmm, y_validation)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [ ]:
model_ug_dmm.save('lib/d2v_dmm_eng')
model_ug_dmm = Doc2Vec.load('lib/d2v_dmm_eng')
model_ug_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)